# Golden-AI Pro: Model Training

Training LSTM + Attention model for XAUUSD prediction.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from src.data.processors.feature_engineering import FeatureEngineer
from src.ai.models.lstm_attention import LSTMAttentionModel

## 1. Load and Prepare Data

In [ ]:
# Load data
df = pd.read_csv('../data/xauusd_h1.csv', parse_dates=['time'], index_col='time')

# Feature engineering
fe = FeatureEngineer()
df_features = fe.create_features(df)

print(f"Features shape: {df_features.shape}")
df_features.head()

## 2. Prepare Sequences for LSTM

In [ ]:
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 0])  # Predict next close
    return np.array(X), np.array(y)

# Select features
feature_cols = ['close', 'volume', 'returns', 'rsi_14', 'macd', 'atr_14']
data = df_features[feature_cols].dropna().values

# Normalize
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Create sequences
seq_length = 60
X, y = create_sequences(data_scaled, seq_length)

print(f"X shape: {X.shape}, y shape: {y.shape}")

## 3. Split Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

## 4. Build and Train Model

In [ ]:
# Create model
model = LSTMAttentionModel(
    sequence_length=seq_length,
    n_features=len(feature_cols),
    lstm_units=[128, 64],
    dropout_rate=0.3
)

model.build_model()
model.model.summary()

In [ ]:
# Prepare targets for multi-output
# Simplified for demonstration
y_dir = np.random.randint(0, 3, size=(len(y_train), 3))  # One-hot encoded
y_entry = y_train
y_conf = np.random.random(size=(len(y_train), 1))

# Train
history = model.train(
    X_train,
    y_dir,
    y_entry,
    y_conf,
    validation_split=0.2,
    epochs=50,
    batch_size=32
)

## 5. Evaluate Model

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(history['loss'], label='Train Loss')
axes[0].plot(history.get('val_loss', []), label='Val Loss')
axes[0].set_title('Model Loss')
axes[0].legend()

axes[1].plot(history.get('direction_accuracy', []), label='Accuracy')
axes[1].set_title('Direction Accuracy')
axes[1].legend()

plt.show()

## 6. Save Model

In [ ]:
model.save('../models/lstm_attention_v1.h5')
print("Model saved!")